In [13]:
import numpy as np
import os , sys
sys.path.append("..")

In [14]:
from Gen_src.diffusion import GaussianDiffusion1D
from Gen_src.model import Unet1D
import torch
import os,sys
# Add the parent directory of Gen_src to sys.path
sys.path.append(os.path.abspath(".."))
sys.path.append(os.path.abspath("../.."))
import torch
import os

# Same architecture as in training
model = Unet1D(
    dim=32,
    dim_mults=(2, 4, 8, 16),
    channels=2,  # X and Y
    dropout=0.1
).cuda()  # or .to(device)

# Create the same diffusion wrapper
diffusion = GaussianDiffusion1D(
    model,
    seq_length=192,      # must match your training setup
    objective='pred_noise',
    timesteps=1000,
    auto_normalize=False
).cuda()  # or .to(device)

# Load checkpoint
checkpoint_path = rf"/home/bardiya/projects/diffusion_air_manifolding/codes/creative-generativeai-diffusion/src/diffusion_notebooks/DIffusion_model_weigths_and_datas/dpp_0.1_autonorm_true_125_from_base_ddpm/model_epoch_124.pt"
model.load_state_dict(torch.load(checkpoint_path, weights_only=True))
model.eval()
print("Loaded model weights from:", checkpoint_path)


Loaded model weights from: /home/bardiya/projects/diffusion_air_manifolding/codes/creative-generativeai-diffusion/src/diffusion_notebooks/DIffusion_model_weigths_and_datas/dpp_0.1_autonorm_true_125_from_base_ddpm/model_epoch_124.pt


In [15]:
from pathlib import Path

DATA_DIR = Path(rf"/home/bardiya/projects/diffusion_air_manifolding/codes/creative-generativeai-diffusion/data/raw")
coord_mm = np.load(DATA_DIR/"coord_min_max.npy")  # [[x_min,y_min],[x_max,y_max]]
x_min,y_min = coord_mm[0]; x_max,y_max = coord_mm[1]

def inv_coords(xs_s):                   # xs_s shape (...,2,192) tensor
    xs_np = xs_s.permute(0,2,1).cpu().numpy()    # -> (B,192,2)
    xs_np[...,0] = xs_np[...,0]*(x_max-x_min) + x_min
    xs_np[...,1] = xs_np[...,1]*(y_max-y_min) + y_min
    return xs_np                                # (B,192,2) numpy


In [16]:
num_to_generate = 2
batch_size      = 2

# generated_scaled = []
# latent = torch.randn((1,2,192)).to("cuda")
# samples1 = diffusion.latent_sample(latent , is_ddim=True)
# samples2 = diffusion.latent_sample(latent , is_ddim=True)
# generated_samples1 = inv_coords(samples1)
# generated_samples2 = inv_coords(samples2)


all_latent = []
all_shapes = []
with torch.no_grad():
    done = 0
    while done < num_to_generate:
        cur = min(batch_size, num_to_generate - done)
        
        latent = torch.randn((cur,2,192)).to("cuda")
        samples = diffusion.latent_sample(latent , is_ddim=True)
        generated_real = inv_coords(samples)
        
        all_latent.append(latent.cpu().detach().numpy())
        all_shapes.append(generated_real)
        done += cur
        print(f"Generated {done}/{num_to_generate}")

sampling loop time step: 100%|██████████| 1000/1000 [00:12<00:00, 77.17it/s]

Generated 2/2


In [17]:
docker_mount_path = "/home/bardiya/projects/diffusion_air_manifolding/codes/Airfoil_MPI_system"
# docker_mount_path = "."

np.save(os.path.join(docker_mount_path , "DB2.npy") , {
    "latents": np.vstack(all_latent),
    "shapes": np.vstack(all_shapes),
    "performances": None
})

In [18]:
sys.path.append("../OpenFoam")
from OpenFoam.Airfoil_simulation_1.ShapeToPerformance import shape_to_performance as STP1
docker_container_id = "78257c051b5b" 
# command = fr"docker exec {docker_container_id} python3 /home/airfoil_UANA/performance_finding.py"
command = (
    f'docker exec {docker_container_id} bash -c "'
    f'source /opt/openfoam5/etc/bashrc && '
    f'cd /home/airfoil_UANA && '
    f'python3 performance_finding.py"'
)

os.system(command)


(2, 192, 2)
8


Error   : Could not find extruded vertex (0.9800665511016924, -0.004483645182613369, 1) in surface 1026
Error   : Could not find extruded vertex (0.9800665511016924, -0.004483645182613369, 1) in surface 1026
Error   : Could not find extruded vertex (0.9800665511016924, -0.004483645182613369, 1) in surface 1026
Error   : Could not find extruded vertex (0.9800665511016924, -0.004483645182613369, 1) in surface 1026
Error   : Could not find extruded vertex (0.9800665511016924, -0.004483645182613369, 1) in surface 1026
Error   : Could not find extruded vertex (0.9800665511016924, -0.004483645182613369, 1) in surface 1026
Error   : Could not find extruded vertex (0.9800665511016924, -0.004483645182613369, 1) in surface 1026
Error   : Unable to recover the edge 1885 (15/452) on GEdge 1000 (on GFace 1026)
Error   : ------------------------------
Error   : Mesh generation error summary
Error   :    43 warnings
Error   :     8 errors
Error   : Check the full log for details
Error   : -----------

core 1:
sample 1:
	Using len 40.000 angle +0.000 
	Resulting freestream vel x,y: 40.0,-0.0
error during mesh creation!
	mesh generation failed, aborting
-1000
1000
/home/airfoil_UANA
/home/airfoil_UANA
core 0:
sample 0:
	Using len 40.000 angle +0.000 
	Resulting freestream vel x,y: 40.0,-0.0
/home/airfoil_UANA/Airfoil_simulation_1/OpenFOAM_0
/home/airfoil_UANA/Airfoil_simulation_1/OpenFOAM_0
0.2474784
0.008937957
0.txt
1.txt
[0 1]
[[ 2.47478e-01  8.93800e-03  0.00000e+00]
 [-1.00000e+03  1.00000e+03  1.00000e+00]]
done!!!


0

In [19]:
performance_path =os.path.join(docker_mount_path , rf"performance.npy")
perfromance = np.load(performance_path,allow_pickle=True)
perfromance

array([[ 2.47478e-01,  8.93800e-03,  0.00000e+00],
       [-1.00000e+03,  1.00000e+03,  1.00000e+00]])

In [20]:
import sys, os
sys.path.append(os.path.abspath(".."))
sys.path.append(os.path.abspath("../.."))
sys.path.append("../surrogate_models")
import torch.nn as nn
import torch.nn.functional as F
import torch

activation_function_list = [torch.tanh, nn.ReLU(), nn.CELU(), nn.LeakyReLU(), nn.ELU(), nn.Hardswish(),torch.tanh, nn.ReLU(), nn.CELU(), nn.LeakyReLU(), torch.tanh]

class MultiLayerPerceptron_forward(nn.Module):
    def __init__(self, input_size, hidden_layers, num_classes, net_n):
        super(MultiLayerPerceptron_forward, self).__init__()
        #################################################################################
        # Initialize the modules required to implement the mlp with given layer   #
        # configuration. input_size --> hidden_layers[0] --> hidden_layers[1] .... -->  #
        # hidden_layers[-1] --> num_classes                                             #
        #################################################################################
        layers = []
        layers.append(nn.Linear((input_size), (hidden_layers[0])))
        for i in range(len(hidden_layers)-1):
            layers.append(nn.Linear((hidden_layers[i]), (hidden_layers[i+1])))

        layers.append(nn.Linear((hidden_layers[len(hidden_layers)-1]), (num_classes)))
        self.layers = nn.Sequential(*layers)
        self.net_n = net_n
        self.hidden_layers = hidden_layers
    def forward(self, x):
        #################################################################################
        # Implement the forward pass computations                                 #
        #################################################################################
        m = activation_function_list[self.net_n]
        for i in range(len(self.hidden_layers)):
            x = self.layers[i](x)
            x = m(x)
        x = (self.layers[len(self.hidden_layers)](x))
        out=x
        return out

#====================================================

class Hybrid_surrogate_MLP(nn.Module):
    def __init__(self, 
                 input_size, 
                 hidden_layers_cl_model ,
                 hidden_layers_cd_model , 
                 net_n_cl=3 , 
                 net_n_cd=3, 
                 path_cl_model = None, 
                 path_cd_model  =  None):
        super(Hybrid_surrogate_MLP, self).__init__()
        self.cl_forward_mlp = MultiLayerPerceptron_forward(input_size , hidden_layers_cl_model ,   num_classes=1  , net_n=net_n_cl)
        self.cd_forward_mlp = MultiLayerPerceptron_forward(input_size , hidden_layers_cd_model ,   num_classes=1  , net_n=net_n_cd)
        if path_cl_model:
            self.cl_forward_mlp.load_state_dict(torch.load(path_cl_model,map_location="cpu"))
        if path_cd_model:
            self.cd_forward_mlp.load_state_dict(torch.load(path_cd_model,map_location="cpu"))

    def forward(self, x):
        #################################################################################
        # Implement the forward pass computations                                 #
        #################################################################################
        """
            x.shape= (batch , 192, 2)
        """
        x = x.reshape(x.shape[0],-1)
        cl = self.cl_forward_mlp(x)
        cd = self.cd_forward_mlp(x)
        return torch.stack([cl,cd],dim=1).squeeze(-1)
 



In [21]:
residv1_best_model = Hybrid_surrogate_MLP(input_size=192 * 2, 
                                 hidden_layers_cd_model=[200,300,300,200],
                                 hidden_layers_cl_model=[150, 200,200,150],
                                 path_cd_model=rf"/home/bardiya/projects/diffusion_air_manifolding/codes/creative-generativeai-diffusion/src/surrogate_models/Surrogate_Model_weigths/just_for_cd_200_300_300_200/mlp_best_model.pt",
                                 path_cl_model=rf"/home/bardiya/projects/diffusion_air_manifolding/codes/creative-generativeai-diffusion/src/surrogate_models/Surrogate_Model_weigths/just_for_cl_150_200_200_150/mlp_best_model.pt"
                                 ).to("cuda")

# Load the checkpoint
# residv1_best_model.load_state_dict(torch.load(checkpoint_dir, map_location=device))
residv1_best_model.eval()
print("Loaded best Residual surrogate model.")

Loaded best Residual surrogate model.


/tmp/ipykernel_3071/298942254.py:55: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.cl_forward_mlp.load_state_dict(torch.load(path_cl_model,map_location="cpu"))
/tmp/ipy

In [24]:
all_shapes[0].shape ,type(all_shapes[0]) 

((2, 192, 2), numpy.ndarray)

In [25]:
temp = torch.from_numpy(all_shapes[0]).to("cuda")
ans = residv1_best_model(temp)
ans

tensor([[0.5923, 0.0068],
        [0.5673, 0.0078]], device='cuda:0', grad_fn=<SqueezeBackward1>)